In [202]:
DATASET_PATH="/home/siddhesh/Desktop/Siddhesh/Job Applications/smart-sense/smart-sense-ML-assessment/data/ML-5/Monthly_data_cmo.csv"
TEST_DATA_PATH="/home/siddhesh/Desktop/Siddhesh/Job Applications/smart-sense/smart-sense-ML-assessment/data/ML-5/CMO_MSP_Mandi.csv"

## Data Pre-processing and EDA

In [203]:
import pandas as pd
import numpy as np

In [204]:
data=pd.read_csv(DATASET_PATH)
data.shape, data.info()
data['APMC'] = data['APMC'].apply(lambda x: x.lower() if isinstance(x, str) else x)
data['Commodity'] = data['Commodity'].apply(lambda x: x.lower() if isinstance(x, str) else x)
data['Month']=data['Month'].apply(lambda x: x.lower() if isinstance(x, str) else x)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62429 entries, 0 to 62428
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   APMC             62429 non-null  object
 1   Commodity        62429 non-null  object
 2   Year             62429 non-null  int64 
 3   Month            62429 non-null  object
 4   arrivals_in_qtl  62429 non-null  int64 
 5   min_price        62429 non-null  int64 
 6   max_price        62429 non-null  int64 
 7   modal_price      62429 non-null  int64 
 8   date             62429 non-null  object
 9   district_name    62429 non-null  object
 10  state_name       62429 non-null  object
dtypes: int64(5), object(6)
memory usage: 5.2+ MB


From the above analysis, we can say that we don't have any null values.

We will predict the model prices, hence remove min_price, max_price column

In the dataset, we can see that the price is determined by the APMC and not by district or state. Also, its important to note that district or state is an larger entity than APMC

In [205]:
data.drop(['min_price','max_price','district_name','state_name','date','Month','arrivals_in_qtl','APMC'],axis=1,inplace=True)
data=data.dropna()
data.isnull().sum()  

Commodity      0
Year           0
modal_price    0
dtype: int64

In [206]:
data.groupby('Commodity').count()

,Year,modal_price
Commodity,,
amba koy,3,3
ambat chuka,34,34
amla,24,24
apple,233,233
arvi,54,54
...,...,...
water melon,153,153
wheat(husked),4097,4097
wheat(unhusked),167,167


### One Hot Encoding

In [207]:
# Our test data doesnt have month, so droping it
# data.drop(['Month','arrivals_in_qtl'],axis=1,inplace=True)
from sklearn.preprocessing import OneHotEncoder  
# enc = OneHotEncoder(handle_unknown='ignore')
# data['Commodity'] = data['Commodity'].astype('category')

# data['new_commodity'] = data['Commodity'].cat.codes

# enc.fit(data[['Commodity']])


# enc_data = pd.DataFrame(enc.fit_transform(
#     data[['new_commodity']]).toarray())
data = pd.get_dummies(data, columns=["Commodity"], prefix = ["commodity"]) 

# data = data.join(enc_data) 

# enc_data

OneHotEncoder(handle_unknown='ignore')

In [208]:
data.head()

,Commodity,Year,modal_price
0,bajri,2015,1463
1,bajri,2016,1875
2,wheat(husked),2015,1731
3,wheat(husked),2016,1999
4,sorgum(jawar),2015,1900


### Determining Features for the dataset

In [182]:
# Using Min/Max Scaler for the dataset
features=data.loc[:, data.columns !='modal_price']
label=data['modal_price']  

#### Here, we can see that there is significant difference in the magnitude of different features, this may cause the undetermined advantage to the features with more magnitude

In [183]:
from sklearn.preprocessing import MinMaxScaler  

scaler=MinMaxScaler()  
features=scaler.fit_transform(features)  

In [150]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score  

train_data, test_data, train_labels, test_labels = train_test_split(features, label, test_size=0.2, random_state=42)  

model=DecisionTreeRegressor()
# model=GradientBoostingRegressor(n_estimators=200, max_depth=3, random_state=0)
# fit=model.fit(train_data,train_labels)  
# y_pred=fit.predict(test_data)  
# r2=r2_score(test_labels,y_pred)

# print("R2 Score: ",r2)


In [225]:
test_data=pd.read_csv(TEST_DATA_PATH)
test_data.drop(['Type','msp_filter'],axis=1,inplace=True)
test_data['commodity'] = test_data['commodity'].apply(lambda x: x.lower() if isinstance(x, str) else x)
test_data.head()

,commodity,year,msprice
0,paddy-unhusked,2012,1250.0
1,rice(paddy-hus),2012,1280.0
2,jowar_hybrid,2012,1500.0
3,sorgum(jawar),2012,1520.0
4,bajri,2012,1175.0


In [226]:
# One hot encoding
oneh = OneHotEncoder(handle_unknown="ignore")
oneh.fit(data[['Commodity']])
# test_data = pd.get_dummies(test_data, columns=["commodity"], prefix = ["commodity"])
test_data=oneh.transform(test_data['commodity'].values.reshape(-1,1)).toarray()
test_data = pd.DataFrame(test_data)

# test_data_1=test_data.loc[:, test_data.columns !='msprice']
# test_labels=test_data['msprice']
# train_data=features
# train_labels=label

# fit=model.fit(train_data,train_labels)  
# y_pred=fit.predict(test_data_1)  
# r2=r2_score(test_labels,y_pred)
# print(r2)
test_data.head()

/home/siddhesh/anaconda3/envs/mlcourse/lib/python3.9/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,0,1,2,3,4,5,6,7,8,9,...,194,195,196,197,198,199,200,201,202,203
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


paddy-unhusked
rice(paddy-hus)
jowar_hybrid
sorgum(jawar)
bajri
maize
ragi_maldandi
pigeon pea (tur)
spilt gerrn gram
split black gram
cotton
cotton_long staple
gr.nut kernels
sunflower
soyabean_black
soyabean
sesamum
niger-seed
wheat(husked)
wheat(unhusked)
barli
gram_yellow
masur_yellow
mustard
safflower
toria_yellow
copra (milling)_yellow
copra (ball)_yellow
coconut
jute_yellow
sugar-cane
paddy-unhusked
rice(paddy-hus)
jowar_hybrid
sorgum(jawar)
bajri
maize
ragi_maldandi
pigeon pea (tur)
spilt gerrn gram
split black gram
cotton
cotton_long staple
gr.nut kernels
sunflower
soyabean_black
soyabean
sesamum
niger-seed
wheat(husked)
wheat(unhusked)
barli
gram_yellow
masur_yellow
mustard
safflower
toria_yellow
copra (milling)_yellow
copra (ball)_yellow
coconut
jute_yellow
sugar-cane
paddy-unhusked
rice(paddy-hus)
jowar_hybrid
sorgum(jawar)
bajri
maize
ragi_maldandi
pigeon pea (tur)
spilt gerrn gram
split black gram
cotton
cotton_long staple
gr.nut kernels
sunflower
soyabean_black
soyabean
